In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../")

from chiseling.dgps.binary_design_rct import BinaryDesignRCT
from chiseling.source.learners.baselearners_general import linreg_learner
from chiseling.source.protocol.IRST import UnitRegistrar, IRSTBinary
from chiseling.source.strategies.chiseling import Chiseling
from chiseling.source.strategies.bonferroni_combiner import BonferroniCombiner

from chiseling.benchmark.benchmark import Benchmark

In [3]:
# Hyperparameters
n = 2000

d = 15
p = 0.5
s = 7

f_opt = "step"
k = 4
tau = 0.4
theta = None
gamma = 0

alpha = 0.05
test_thresh = 0
random_seed = 80

In [4]:
strategy_settings = {"n_burn_in": 0.2,
                     "margin_width": 1,
                     "reveal_batch_size": max(1, int(n * 0.01)),
                     "refit_batch_size": max(1, int(n * 0.05)),
                     "test_thresh": test_thresh,
                     "learner": linreg_learner,
                     "alpha": alpha,
                     "random_seed": random_seed}

In [5]:
sampler = BinaryDesignRCT(d, p, s, k=k, tau=tau, theta=theta, gamma=gamma, f_opt=f_opt,
                          random_seed=random_seed)
X, Y = sampler.sample(n)

In [6]:
# Run chiseling
strategy = Chiseling(X=X,
                     Y=Y,
                     **strategy_settings)
strategy.run_strategy()

In [7]:
region = strategy.protocol.get_rejected_region()

In [8]:
region is not None

True

In [9]:
strategy.strategy.metrics["FINAL"]

{'rejected': True,
 'spent_alpha': np.float64(0.001480384900074072),
 'remaining_alpha': np.float64(0.048591549295774694),
 'n_left_in_region': 1411,
 'region_mass_estimate': np.float64(0.8818750000000003)}

In [10]:
strategy_settings = {"margin_width": 1,
                     "reveal_batch_size": max(1, int(n * 0.01)),
                     "refit_batch_size": max(1, int(n * 0.05)),
                     "test_thresh": test_thresh,
                     "learner": linreg_learner,
                     "alpha": alpha,
                     "random_seed": random_seed}

In [11]:
# Run chiseling
strategy = BonferroniCombiner(X=X,
                              Y=Y,
                              strategy="Chiseling",
                              strategy_settings=strategy_settings,
                              alpha=alpha,
                              random_seed=random_seed)
strategy.run_strategy()

In [12]:
strategy.rejected_region_split

np.float64(0.8)

In [13]:
strategy.rejected_region

### Benchmark module tests

In [22]:
# Hyperparameters
n = 2000

d = 15
p = 0.5
s = 7

# f_opt = "linear"
# k = None
# tau = 0
# theta = 0.02
# gamma = None

f_opt = "step"
k = 4
tau = 0.2
theta = None
gamma = 0.1
gamma = 0

alpha = 0.05
random_seed = 42

In [23]:
# settings = {"task_id": 0,
#             "dgp": "BinaryDesignRCT",
#              "n": n,
#              "d": d,
#              "p": p,
#              "s": s,
#              "k": k,
#              "tau": tau,
#              "theta": theta,
#              "gamma": gamma,
#              "f_opt": f_opt,
#              "ipw_transform": True,
#              "alpha": alpha,
#              "random_seed": random_seed,
#              "strategy": "TTestStrategy",
#              "test_thresh": 0,
#              "learner": None,
#              "n_sims": 100}

settings = {"task_id": 0,
            "dgp": "BinaryDesignRCT",
             "n": n,
             "d": d,
             "p": p,
             "s": s,
             "k": k,
             "tau": tau,
             "theta": theta,
             "gamma": gamma,
             "f_opt": f_opt,
             "ipw_transform": True,
             "alpha": alpha,
             "random_seed": random_seed,
             "strategy": "Chiseling",
             "n_burn_in": 0.05,
             "margin_width": 0.5,
             "reveal_batch_size": max(1, int(0.01 * n)),
             "refit_batch_size": max(1, int(0.05 * n)),
             "test_thresh": 0,
             "learner": "linreg_learner",
             "use_learner_weights": False,
             "n_sims": 100}

In [24]:
benchmark = Benchmark(settings)
benchmark.simulate_batch()
benchmark.simulation_results_df.head()

,task_id,sim_id,rejected,region_mass,subgroup_mean,subgroup_utility,region_mass_se,subgroup_mean_se,subgroup_utility_se
0,0,0,True,0.80271,0.099206,0.079634,0.001258,0.000353,0.000310
1,0,1,True,0.49400,0.114684,0.056654,0.001581,0.000445,0.000285
2,0,2,True,0.79968,0.106485,0.085154,0.001266,0.000353,0.000313
3,0,3,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
4,0,4,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000


In [25]:
benchmark.simulation_results_df.rejected.mean(axis=0)

np.float64(0.63)